Requirement for charge bias line
1. proper voltage range, a few period of 2e is fine. 
2. minimum voltage noise at chip level 
    - reduce bandwidth : use LPF filters like mini-circuit and Eccosorb
    - voltage divider : to reduce both signal and noise and to get desired voltage resolution
3. power dissipation on attenuator needs to be lower than cooling power of each temperature stage.

# Functions and notations

In [85]:
import numpy as np
from scipy.constants import h,e,hbar,k

# Vp=voltage amplitude, Vpp=2*Vp
dBm_to_W = lambda x: pow(10, x/10) * 1e-3
dBm_to_Vp = lambda x: np.sqrt(dBm_to_W(x)*50)*np.sqrt(2)
Vp_to_dBm = lambda x: 10*np.log10(x**2/50/2/1e-3) 

# class Attenuator(object):
#     def __init__(self, att_in_dB):
#         self.att = att_in_dB
    
def att_power_dissipated(Pin_dBm=0, Att_dB=0):
    """ power dissipated on attenuator in Watt
        Args:
            Pin_dBm : input power in dBm
            Att_dB: attenuation value >= 0
        Return:
            Power dissipation in Watt
        """

    P_dissipated = dBm_to_W(Pin_dBm) - dBm_to_W(Pin_dBm-Att_dB) # in Watt

    return P_dissipated

# Attenuation for microwave line to bias-T for RF-driving JJ

In [83]:
# calculate total attenuation need to reach Ic
# Model: Thevnin voltage source (V_Th) and RLC circuit in series.
# Thevnin-equivalent circuit provides V_Th with 50 Ohm output impedance. The load connected bewteen
# 50 ohm and gnd is a charge bias capacitor and Josephson inductor in series. 

import matplotlib.pyplot as plt
import numpy as np

C_bias = 0.1e-15
Lj = 22e-9 
omega =  5e9 * 2 * np.pi
ZL = 1j*omega*Lj
ZC = 1/(1j*omega*C_bias)
Ic = 15e-9 # critical current

# What Thevonin voltage V_Th is required to get Ic on L
V_Th = Ic * (50+ ZL + ZC)
print(f'|V_Th| = {np.absolute(V_Th):.3e} V')
print(f'|V_source=V_Th/2| = {np.absolute(V_Th/2):.3e} V')


|V_Th| = 4.750e-03 V
|V_source=V_Th/2| = 2.375e-03 V


In [3]:
Att = 42 # dB
Pin = 0 # in dBm
Pout = Pin - Att
# Vin = 1   # 1Vp = 10dBm
# Vout = Vin * pow(10, -(Att/20))
Vout = dBm_to_Vp(Pout)
print(f'|Vout| = {Vout:.3e} V')

# 20 dB at 4K, 3dB still, 10 dB at MC

|Vout| = 2.512e-03 V


In [4]:
print(f'ZL = {ZL.imag:.3f}j Ohm, ZC = {ZC.imag:.3f}j Ohm')

ZL = 691.150j Ohm, ZC = -318309.886j Ohm


# Attenuation for DC line to bias-T

In [80]:
# attenuation for DC
# With open load to a attunator, power dissipation and voltage division are the same for T and Pi type
# https://www.microwaves101.com/calculators/858-attenuator-calculator

# For T-type attenuator
Vin = 0.051 # Vp in volt
V_resol = 1e-3
dB = 26 # attenation in dB

Z0 = 50
R1 = Z0 * ((10**(dB / 20) - 1) / (10**(dB / 20) + 1))
R2 = 2 * Z0 * (10**(dB / 20) / (10**(dB / 10) - 1))

V_ratio = R2 / (R1 + R2)
Vout = Vin * V_ratio
Vout_resol = V_resol * V_ratio
P_diss = Vin**2/ (R1+R2)

print('T-attenuator')
print(f'Vin = {Vin:.3e} V')
print(f'Vout = {Vout:.3e} V')
print(f'Vout_resol = {Vout_resol:.3e} V')
print(f'Vout/Vin ratio = {Vout/Vin:.3f}')
print(f'Power dissipated = {P_diss:.3e} W')

T-attenuator
Vin = 5.100e-02 V
Vout = 5.099e-03 V
Vout_resol = 9.999e-05 V
Vout/Vin ratio = 0.100
Power dissipated = 5.176e-05 W


In [81]:
Vp_to_dBm(2.38e-3)

-42.46846085886976

In [92]:
# noise voltage
T = 4
B= 100e6
R=60

V_rms = np.sqrt(4*k*B*T*R)
print(f'V_rms = {V_rms:.3e} V')


V_rms = 1.151e-06 V
